# 蒙特卡洛方法核心设计解析

## 📚 教程目标

本教程深入解析 `MC_Basic.ipynb` 和 `MC_Epsilon_Greedy.ipynb` 中的核心设计理念，帮助你理解：

- **Why (为什么)**：设计决策的背后原因
- **What (是什么)**：核心概念和数据结构
- **How (怎么做)**：函数间的关联和工作流程

我们将通过简洁的、解耦的代码示例，逐步揭示项目设计的精妙之处。

---

## 第一部分：状态表示设计 - `_pos_to_state` 和 `_state_to_pos`

### 🤔 Why：为什么需要状态转换？

在网格世界中，我们有两种表示智能体位置的方式：

1. **二维坐标 (row, col)**：直观，易于可视化和环境交互
2. **一维索引 state**：适合用作数组索引，存储 Q 值表

**核心问题**：如何在这两种表示之间高效转换？

**设计决策**：使用行优先的线性映射，保证：
- ✅ 双向转换的唯一性
- ✅ O(1) 时间复杂度
- ✅ 连续性（相邻位置 → 接近的状态索引）

In [ ]:
# 导入必要的库
import numpy as np
from typing import Annotated
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('Agg')

print("✅ 库导入完成")

### 💡 What：核心映射公式

对于一个 `size × size` 的网格：

**位置 → 状态**（行优先编号）：
```
state = row × size + col
```

**状态 → 位置**（逆向解码）：
```
row = state // size  （整除）
col = state % size   （取余）
```

**为什么这样设计？**

这是一个经典的**二维数组线性化**技巧：
- 将二维矩阵按行展开成一维数组
- 每一行占据 `size` 个连续位置
- 通过整除和取余可以完美还原原始位置

In [ ]:
# 示例：4×4 网格的状态编号可视化
def visualize_state_mapping(
    size: Annotated[int, "网格的边长"] = 4,
) -> None:
    """可视化二维位置到一维状态的映射关系"""
    
    print(f"{'=' * 50}")
    print(f"  {size}×{size} 网格的状态编号（行优先）")
    print(f"{'=' * 50}")
    print()
    
    # 打印表头
    print("     ", end="")
    for col in range(size):
        print(f"col{col:2d}  ", end="")
    print()
    print("    " + "-" * (size * 7))
    
    # 打印每一行
    for row in range(size):
        print(f"row{row} | ", end="")
        for col in range(size):
            state = row * size + col
            print(f"  {state:2d}   ", end="")
        print()
    
    print()
    print("💡 观察：")
    print("   - 每行的状态索引是连续的")
    print("   - 从左到右、从上到下递增")
    print(f"   - 总共有 {size * size} 个状态（0 到 {size * size - 1}）")
    print()
    print("🎯 实际意义：")
    print("   - Q[state, action] 可以直接用状态索引访问")
    print("   - policy[state] 也能高效存储每个状态的策略")

visualize_state_mapping(4)

### 🔧 How：函数实现与验证

In [ ]:
class StateConverter:
    """状态转换器：演示位置-状态互转的核心逻辑"""
    
    def __init__(self, size: Annotated[int, "网格的边长"] = 4):
        self.size = size
    
    def pos_to_state(
        self, 
        pos: Annotated[np.ndarray, "二维位置坐标 [row, col]"],
    ) -> Annotated[int, "对应的一维状态索引"]:
        """将二维位置转换为一维状态索引
        
        核心公式：state = row × size + col
        
        为什么这样设计？
        - 行优先编号保证了空间连续性
        - 相邻的位置对应相近的状态索引
        - 便于在数组中存储和访问 Q 值
        """
        return pos[0] * self.size + pos[1]
    
    def state_to_pos(
        self, 
        state: Annotated[int, "一维状态索引"],
    ) -> Annotated[np.ndarray, "对应的二维位置坐标 [row, col]"]:
        """将一维状态索引转换为二维位置
        
        核心公式：
        - row = state // size  （整除得到行号）
        - col = state % size   （取余得到列号）
        
        数学原理：
        如果 state = row × size + col
        那么 state // size = row（因为 col < size）
        且   state % size = col（取余操作）
        """
        row = state // self.size
        col = state % self.size
        return np.array([row, col])

# 创建转换器并测试
converter = StateConverter(size=4)

print("🧪 测试状态转换的正确性：")
print()

# 测试 1：位置 → 状态
test_positions = [
    np.array([0, 0]),  # 左上角
    np.array([1, 3]),  # 目标位置（MC 示例中的目标）
    np.array([2, 1]),  # 中间某个位置
    np.array([3, 3]),  # 右下角
]

for pos in test_positions:
    state = converter.pos_to_state(pos)
    print(f"位置 {pos} → 状态 {state}")

print()
print("🔄 测试双向转换的一致性：")
print()

# 测试 2：往返转换验证
for state in [0, 7, 10, 15]:
    pos = converter.state_to_pos(state)
    back_to_state = converter.pos_to_state(pos)
    print(f"状态 {state:2d} → 位置 {pos} → 状态 {back_to_state:2d}  {'✅' if state == back_to_state else '❌'}")

print()
print("💡 关键洞察：")
print("   1. 转换是双射（一一对应），保证了唯一性")
print("   2. 无论正向还是反向，转换都是 O(1) 时间复杂度")
print("   3. 这种设计让 Q[state, action] 可以高效访问")

### 🎯 设计优势总结

| 方面 | 优势 | 说明 |
|------|------|------|
| **存储效率** | Q 值表只需二维数组 | `Q[num_states, num_actions]` 而非三维 |
| **访问速度** | O(1) 直接索引 | 无需哈希或查找 |
| **空间连续性** | 相邻位置 → 相近索引 | 有利于缓存友好性 |
| **可扩展性** | 适用于任意大小网格 | 公式统一，无特殊情况 |

---

## 第二部分：策略表示 - `np.random.choice(4, p=policy[state])`

### 🤔 Why：为什么用概率分布表示策略？

在强化学习中，策略可以是：
1. **确定性策略**：每个状态固定选择一个动作
2. **随机策略**：每个状态以一定概率选择动作

**设计决策**：使用概率分布表示策略，因为：
- ✅ **统一表示**：确定性策略是特殊情况（某个动作概率为 1）
- ✅ **支持探索**：ε-贪心策略需要概率表示
- ✅ **数学优雅**：与策略梯度等高级方法兼容

### 💡 What：策略矩阵的数据结构

策略表示为一个二维数组：
```python
policy.shape = (num_states, num_actions)
policy[s, a] = 在状态 s 选择动作 a 的概率
```

**约束条件**：
- 对于每个状态 s：`sum(policy[s, :]) = 1.0`（概率和为 1）
- 所有概率非负：`policy[s, a] >= 0`

In [ ]:
def demonstrate_policy_types(
    num_states: Annotated[int, "状态空间大小"] = 16,
    num_actions: Annotated[int, "动作空间大小"] = 4,
) -> None:
    """演示不同类型的策略表示"""
    
    print("📊 策略类型示例（以状态 7 为例）")
    print("=" * 60)
    
    # 1. 确定性策略（贪婪策略）
    greedy_policy = np.zeros((num_states, num_actions))
    best_action = 2  # 假设状态 7 的最优动作是 2（左）
    greedy_policy[7, best_action] = 1.0
    
    print("\n1️⃣ 确定性策略（完全贪婪）：")
    print(f"   policy[7] = {greedy_policy[7]}")
    print(f"   → 总是选择动作 {best_action}（左）")
    print(f"   → 概率和验证：{greedy_policy[7].sum():.1f} ✅")
    
    # 2. 均匀随机策略
    uniform_policy = np.ones((num_states, num_actions)) / num_actions
    
    print("\n2️⃣ 均匀随机策略（完全探索）：")
    print(f"   policy[7] = {uniform_policy[7]}")
    print(f"   → 每个动作概率相同（各 25%）")
    print(f"   → 概率和验证：{uniform_policy[7].sum():.1f} ✅")
    
    # 3. ε-贪心策略（平衡探索与利用）
    epsilon = 0.1
    epsilon_greedy = np.ones((num_states, num_actions)) * epsilon / num_actions
    epsilon_greedy[7, best_action] += (1.0 - epsilon)
    
    print(f"\n3️⃣ ε-贪心策略（ε={epsilon}）：")
    print(f"   policy[7] = {epsilon_greedy[7]}")
    print(f"   → 最优动作概率：{epsilon_greedy[7, best_action]:.2f} (90% + 2.5%)")
    print(f"   → 其他动作概率：{epsilon_greedy[7, 0]:.2f} (2.5% 各)")
    print(f"   → 概率和验证：{epsilon_greedy[7].sum():.4f} ✅")
    
    print("\n💡 关键洞察：")
    print("   - 确定性策略：纯利用，可能陷入局部最优")
    print("   - 均匀策略：纯探索，学习效率低")
    print("   - ε-贪心：平衡探索与利用，实用性最强")

demonstrate_policy_types()

### 🔧 How：`np.random.choice` 的精妙之处

**核心代码**：
```python
action = np.random.choice(4, p=policy[state])
```

**这一行代码的精妙设计**：

1. **参数 `4`**：动作空间大小（0, 1, 2, 3 对应 上、下、左、右）
2. **参数 `p=policy[state]`**：在状态 `state` 下各动作的概率分布
3. **返回值**：按概率随机选择的一个动作索引

**为什么这样设计很巧妙？**
- ✅ **统一接口**：无论是贪婪策略还是 ε-贪心，都用同一行代码
- ✅ **自动采样**：NumPy 自动处理概率分布采样，无需手动实现
- ✅ **正确性保证**：只要 `policy[state]` 是有效概率分布，就能正确采样

In [ ]:
def demonstrate_action_sampling(
    num_samples: Annotated[int, "采样次数"] = 10000,
) -> None:
    """演示动作采样过程及其统计特性"""
    
    print("🎲 动作采样演示")
    print("=" * 60)
    
    # 定义一个 ε-贪心策略（状态 7，最优动作是 2）
    epsilon = 0.1
    num_actions = 4
    best_action = 2
    
    # 构建策略
    policy_state_7 = np.ones(num_actions) * epsilon / num_actions
    policy_state_7[best_action] += (1.0 - epsilon)
    
    print(f"\n策略设置（状态 7）：")
    print(f"   ε = {epsilon}")
    print(f"   最优动作 = {best_action}（左）")
    print(f"   策略概率分布：{policy_state_7}")
    print()
    
    # 方法 1：使用 np.random.choice（推荐）
    print("1️⃣ 使用 np.random.choice 采样：")
    action_counts_method1 = np.zeros(num_actions)
    for _ in range(num_samples):
        action = np.random.choice(num_actions, p=policy_state_7)
        action_counts_method1[action] += 1
    
    print(f"   采样 {num_samples} 次的结果：")
    action_names = ['上', '下', '左', '右']
    for a in range(num_actions):
        actual_prob = action_counts_method1[a] / num_samples
        expected_prob = policy_state_7[a]
        print(f"   动作 {a}（{action_names[a]}）: 实际={actual_prob:.4f}, 期望={expected_prob:.4f}, 误差={abs(actual_prob - expected_prob):.4f}")
    
    print("\n💡 为什么使用 np.random.choice？")
    print("   ✅ 代码简洁：一行代码完成采样")
    print("   ✅ 性能优化：C 语言实现，速度快")
    print("   ✅ 数值稳定：自动处理浮点数精度问题")
    print("   ✅ 可读性强：意图清晰，易于理解")

demonstrate_action_sampling()

### 🎯 从 Q 值到策略的转换模式

#### 模式 1：创建 ε-贪心策略的公式推导

**目标**：对于每个状态，最优动作有更高概率，其他动作保留探索概率

**数学表达**：
$$
\pi(a|s) = \begin{cases}
1 - \epsilon + \frac{\epsilon}{|A|} & \text{if } a = \arg\max Q(s,a) \\
\frac{\epsilon}{|A|} & \text{otherwise}
\end{cases}
$$

**代码实现**：

In [ ]:
def create_epsilon_greedy_policy_explained(
    Q: Annotated[np.ndarray, "状态-动作价值表"],
    epsilon: Annotated[float, "探索概率"],
    num_states: Annotated[int, "状态数量"],
    num_actions: Annotated[int, "动作数量"],
) -> Annotated[np.ndarray, "ε-贪心策略矩阵"]:
    """创建 ε-贪心策略（详细注释版本）"""
    
    # 步骤 1：初始化所有动作的基础探索概率
    # 每个动作都有 ε/|A| 的基础概率
    policy = np.ones((num_states, num_actions)) * epsilon / num_actions
    print(f"步骤 1：基础探索概率 = {epsilon}/{num_actions} = {epsilon/num_actions:.4f}")
    
    # 步骤 2：为最优动作增加额外的利用概率
    for s in range(num_states):
        best_a = np.argmax(Q[s])  # 找到 Q 值最大的动作
        # 最优动作获得 (1-ε) 的额外概率
        policy[s, best_a] += (1.0 - epsilon)
    
    print(f"步骤 2：最优动作额外概率 = {1.0 - epsilon:.4f}")
    print(f"\n最终概率分布：")
    print(f"   - 最优动作：{epsilon/num_actions:.4f} + {1.0-epsilon:.4f} = {epsilon/num_actions + 1.0-epsilon:.4f}")
    print(f"   - 其他动作：{epsilon/num_actions:.4f}")
    print(f"   - 概率和验证：{epsilon/num_actions * (num_actions-1) + (epsilon/num_actions + 1.0-epsilon):.4f}")
    
    return policy

# 示例：创建一个简单的 Q 值表
Q_example = np.array([
    [0.1, 0.3, 0.8, 0.2],  # 状态 0：动作 2 最优
    [0.5, 0.2, 0.1, 0.9],  # 状态 1：动作 3 最优
])

print("\n🧪 ε-贪心策略创建演示：")
print("=" * 60)
policy_example = create_epsilon_greedy_policy_explained(
    Q_example, epsilon=0.1, num_states=2, num_actions=4
)

print("\n生成的策略：")
print("状态 0:", policy_example[0])
print("状态 1:", policy_example[1])

---

## 第三部分：函数关联与工作流程

### 🔗 完整的数据流与函数调用链

In [ ]:
def illustrate_complete_workflow() -> None:
    """图解蒙特卡洛方法的完整工作流程"""
    
    workflow = """
╔════════════════════════════════════════════════════════════════════╗
║           蒙特卡洛方法完整工作流程（函数调用链）                   ║
╚════════════════════════════════════════════════════════════════════╝

┌────────────────────────────────────────────────────────────────────┐
│  阶段 1：初始化                                                    │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│  Q ← zeros(num_states, num_actions)    # Q 值表初始化            │
│  policy ← uniform_policy()             # 策略初始化              │
│  returns ← defaultdict(list)           # 回报记录               │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘

┌────────────────────────────────────────────────────────────────────┐
│  阶段 2：生成回合（Episode Generation）                           │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│  1. reset() → 初始化环境                                          │
│     ├─ 设置起始位置（随机或指定）                                 │
│     └─ pos_to_state(start_pos) → state  【状态转换】             │
│                                                                    │
│  2. 循环生成轨迹：                                                │
│     while not terminated:                                          │
│        ├─ action = np.random.choice(4, p=policy[state]) 【采样】  │
│        │    └─ 根据策略概率选择动作                               │
│        │                                                           │
│        ├─ next_state, reward = step(action)  【环境交互】        │
│        │    ├─ new_pos = pos + direction                          │
│        │    ├─ reward = rewards[new_pos]                          │
│        │    └─ next_state = pos_to_state(new_pos) 【状态转换】   │
│        │                                                           │
│        └─ episode.append((state, action, reward))                 │
│           state = next_state                                       │
│                                                                    │
│  3. 返回完整回合：[(s₀,a₀,r₁), (s₁,a₁,r₂), ..., (sₜ,aₜ,rₜ₊₁)]  │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘

┌────────────────────────────────────────────────────────────────────┐
│  阶段 3：计算回报（Return Calculation）                           │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│  G ← 0                                                             │
│  for (s, a, r) in reversed(episode):   # 从后向前遍历            │
│      G = r + γ × G                      # 累积折扣回报            │
│      returns[(s,a)].append(G)           # 记录该 (s,a) 的回报    │
│                                                                    │
│  公式：Gₜ = Rₜ₊₁ + γRₜ₊₂ + γ²Rₜ₊₃ + ...                         │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘

┌────────────────────────────────────────────────────────────────────┐
│  阶段 4：更新 Q 值（Q-Value Update）                              │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│  for each (s, a) in episode:                                       │
│      Q[s, a] = mean(returns[(s, a)])  # 平均所有回报             │
│                                                                    │
│  核心思想：Q(s,a) ≈ 从 (s,a) 开始的期望回报                      │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘

┌────────────────────────────────────────────────────────────────────┐
│  阶段 5：策略改进（Policy Improvement）                           │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│  方式 1：贪婪策略（MC Basic）                                     │
│    for each state s:                                               │
│        best_a = argmax(Q[s])                                       │
│        policy[s, best_a] = 1.0                                     │
│        policy[s, others] = 0.0                                     │
│                                                                    │
│  方式 2：ε-贪心策略（MC Epsilon-Greedy）                         │
│    policy = ones(num_states, num_actions) × ε / num_actions      │
│    for each state s:                                               │
│        best_a = argmax(Q[s])                                       │
│        policy[s, best_a] += (1 - ε)                               │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘

┌────────────────────────────────────────────────────────────────────┐
│  关键函数的输入输出关系                                            │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│  _pos_to_state:                                                    │
│    输入：np.array([row, col])                                      │
│    输出：int (state index)                                         │
│    用途：环境 → 算法的桥梁                                        │
│                                                                    │
│  _state_to_pos:                                                    │
│    输入：int (state index)                                         │
│    输出：np.array([row, col])                                      │
│    用途：算法 → 环境的桥梁                                        │
│                                                                    │
│  np.random.choice:                                                 │
│    输入：num_actions, p=policy[state]                              │
│    输出：int (action index)                                        │
│    用途：策略 → 动作的采样                                        │
│                                                                    │
│  generate_episode:                                                 │
│    输入：policy, start_state (可选)                                │
│    输出：[(s, a, r), ...] 轨迹                                     │
│    用途：数据收集                                                  │
│                                                                    │
│  calculate_return:                                                 │
│    输入：episode, gamma                                            │
│    输出：float (discounted return)                                 │
│    用途：评估轨迹价值                                              │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘

💡 核心洞察：
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
1. 状态转换函数是桥梁：连接直观的二维空间与高效的一维索引
2. 概率策略是核心：统一了探索与利用，简化了采样逻辑
3. 回合生成是基础：通过与环境交互收集经验数据
4. Q 值更新是关键：将经验转化为知识（动作价值估计）
5. 策略改进是目标：基于 Q 值不断优化决策
    """
    
    print(workflow)

illustrate_complete_workflow()

### 🔍 MC Basic vs MC Epsilon-Greedy 对比

In [ ]:
def compare_mc_algorithms() -> None:
    """对比两种蒙特卡洛算法的差异"""
    
    comparison = """
╔════════════════════════════════════════════════════════════════════╗
║              MC Basic vs MC Epsilon-Greedy 核心差异对比            ║
╚════════════════════════════════════════════════════════════════════╝

┌─────────────────┬──────────────────────┬──────────────────────┐
│   对比维度      │    MC Basic          │  MC Epsilon-Greedy   │
├─────────────────┼──────────────────────┼──────────────────────┤
│ 回合起点        │ 必须从每个 (s,a) 开始│ 随机起点即可         │
│                 │ （探索性起点）       │                      │
├─────────────────┼──────────────────────┼──────────────────────┤
│ 探索方式        │ 起点保证探索         │ ε-贪心策略保证探索   │
│                 │                      │                      │
├─────────────────┼──────────────────────┼──────────────────────┤
│ 策略类型        │ 确定性贪婪策略       │ 随机策略（概率分布） │
│                 │ policy[s, a] ∈ {0,1} │ policy[s, a] ∈ [0,1] │
├─────────────────┼──────────────────────┼──────────────────────┤
│ 评估-改进分离   │ 显式分离             │ 隐式融合             │
│                 │ 先评估所有(s,a)      │ 每个回合都在优化     │
│                 │ 再统一改进策略       │                      │
├─────────────────┼──────────────────────┼──────────────────────┤
│ 采样效率        │ 较低                 │ 较高                 │
│                 │ 每个(s,a)都需多次采样│ 经验复用更充分       │
├─────────────────┼──────────────────────┼──────────────────────┤
│ 实用性          │ 理论教学价值高       │ 实际应用更广泛       │
│                 │ 清晰展示算法结构     │ 更符合现实约束       │
├─────────────────┼──────────────────────┼──────────────────────┤
│ 收敛速度        │ 较快（如果能探索全） │ 较慢但更稳定         │
│                 │                      │                      │
└─────────────────┴──────────────────────┴──────────────────────┘

代码对比示例：
═══════════════════════════════════════════════════════════════════

【MC Basic】
────────────────────────────────────────────────────────────────
# 1. 必须从每个 (s,a) 开始生成回合
for state in range(num_states):
    for action in range(num_actions):
        episode = env.generate_episode(
            start_state=state,      # 指定起点
            start_action=action,    # 指定首个动作
            policy=policy
        )
        G = calculate_return(episode, gamma)
        returns[(state, action)].append(G)

# 2. 更新 Q 值
for (s, a) in all_state_action_pairs:
    Q[s, a] = mean(returns[(s, a)])

# 3. 策略改进（贪婪）
for s in range(num_states):
    best_a = argmax(Q[s])
    policy[s, best_a] = 1.0  # 确定性策略
    policy[s, others] = 0.0

【MC Epsilon-Greedy】
────────────────────────────────────────────────────────────────
# 1. 从随机起点生成回合
for episode_num in range(num_episodes):
    # 策略自动包含最新的 Q 值
    policy = create_epsilon_greedy_policy(Q, epsilon)
    
    # 随机起点
    episode = env.generate_episode(policy)  # 无需指定起点
    
    # 2. 更新 Q 值（首次访问）
    G = 0
    visited = set()
    for (s, a, r) in reversed(episode):
        G = r + gamma * G
        if (s, a) not in visited:
            returns[(s, a)].append(G)
            Q[s, a] = mean(returns[(s, a)])
            visited.add((s, a))
    
    # 3. 策略隐式改进（下一轮自动使用新 Q 值）

核心差异总结：
═══════════════════════════════════════════════════════════════════

MC Basic:
  优点：算法结构清晰，评估-改进明确分离
  缺点：需要探索性起点，实际应用中难以满足
  适用：教学演示，理解蒙特卡洛方法的基本原理

MC Epsilon-Greedy:
  优点：无需特殊起点，通过策略本身保证探索
  缺点：需要更多回合才能收敛
  适用：实际应用，更符合现实约束

共同点：
  - 都是无模型（model-free）方法
  - 都基于完整回合进行学习
  - 都使用蒙特卡洛估计 Q 值
  - 都保证收敛到最优策略（在一定条件下）
    """
    
    print(comparison)

compare_mc_algorithms()

---

## 总结：设计哲学与最佳实践

### 🎯 核心设计原则

1. **抽象的力量**
   - 状态转换：隐藏实现细节，提供简洁接口
   - 策略表示：统一确定性和随机策略
   - 回合生成：封装复杂的交互逻辑

2. **单一职责**
   - 每个函数做好一件事
   - `_pos_to_state`：只负责转换
   - `generate_episode`：只负责生成轨迹
   - `calculate_return`：只负责计算回报

3. **数据驱动**
   - Q 值表：存储学到的知识
   - 策略矩阵：编码决策规则
   - 回报记录：追踪学习历史

### 💡 关键洞察

- **好的设计是看不见的**：使用时感觉自然，那就是好设计
- **抽象的价值**：正确的抽象让复杂问题变简单
- **统一的优雅**：统一的接口让代码简洁优美
- **类型注解的力量**：自文档化，减少错误，易于维护

### 🚀 扩展建议

1. **改进探索策略**
   - 递减 ε：`ε = ε_start × decay^episode`
   - UCB 探索：选择不确定性高的动作
   - 好奇心驱动：奖励未探索的状态

2. **优化采样效率**
   - 经验回放：重复利用历史回合
   - 重要性采样：纠正 off-policy 偏差
   - n-step returns：平衡偏差和方差

3. **函数逼近**
   - 线性函数：Q(s,a) ≈ w^T φ(s,a)
   - 神经网络：深度 Q 网络 (DQN)
   - 特征工程：手工设计状态特征

### 🎓 下一步

现在你已经深入理解了蒙特卡洛方法的设计精髓，可以：

1. 回到原始代码，重新审视每个设计决策
2. 尝试实现一个变种（如递减 ε 或 n-step MC）
3. 思考如何将这些模式应用到其他强化学习算法

**记住**：理解设计的 Why、What、How，比记住代码本身更重要！

---

**Happy Learning! 🚀**